## Getting Started with SpanMarker
[SpanMarker](https://github.com/tomaarsen/SpanMarkerNER) is an accessible yet powerful Python module for training Named Entity Recognition models.

In this notebook, we'll have a look at how to train an NER model using SpanMarker.

### Setup
First of all, the `span_marker` Python module needs to be installed. If we want to use [Weights and Biases](https://wandb.ai/) for logging, we can install `span_marker` using the `[wandb]` extra.

In [ ]:
%pip install span_marker
# %pip install span_marker[wandb]

### Loading the dataset
For this example, we'll load the challenging [FewNERD supervised dataset](https://huggingface.co/datasets/DFKI-SLT/few-nerd) from the Hugging Face hub using 🤗 Datasets.

In [2]:
from datasets import load_dataset

dataset = load_dataset("DFKI-SLT/few-nerd", "supervised")
dataset

100%|██████████| 3359329/3359329 [00:09<00:00, 342056.99it/s]


100%|██████████| 482037/482037 [00:01<00:00, 346172.32it/s]


100%|██████████| 958765/958765 [00:02<00:00, 346564.24it/s]


Dataset few-nerd downloaded and prepared to .... Subsequent calls will reuse this data.


DatasetDict({
    train: Dataset({
        features: ['id', 'tokens', 'ner_tags', 'fine_ner_tags'],
        num_rows: 131767
    })
    validation: Dataset({
        features: ['id', 'tokens', 'ner_tags', 'fine_ner_tags'],
        num_rows: 18824
    })
    test: Dataset({
        features: ['id', 'tokens', 'ner_tags', 'fine_ner_tags'],
        num_rows: 37648
    })
})

Let's inspect some samples to get a feel for the data.

In [3]:
for sample in dataset["train"].select(range(3)):
    print(sample)

{'id': '0', 'tokens': ['Paul', 'International', 'airport', '.'], 'ner_tags': [0, 0, 0, 0], 'fine_ner_tags': [0, 0, 0, 0]}
{'id': '1', 'tokens': ['It', 'starred', 'Hicks', "'s", 'wife', ',', 'Ellaline', 'Terriss', 'and', 'Edmund', 'Payne', '.'], 'ner_tags': [0, 0, 7, 0, 0, 0, 7, 7, 0, 7, 7, 0], 'fine_ner_tags': [0, 0, 51, 0, 0, 0, 50, 50, 0, 50, 50, 0]}
{'id': '2', 'tokens': ['``', 'Time', '``', 'magazine', 'said', 'the', 'film', 'was', '``', 'a', 'multimillion', 'dollar', 'improvisation', 'that', 'does', 'everything', 'but', 'what', 'the', 'title', 'promises', "''", 'and', 'suggested', 'that', '``', 'writer', 'George', 'Axelrod', '(', '``', 'The', 'Seven', 'Year', 'Itch', '``', ')', 'and', 'director', 'Richard', 'Quine', 'should', 'have', 'taken', 'a', 'hint', 'from', 'Holden', '[', "'s", 'character', 'Richard', 'Benson', ']', ',', 'who', 'writes', 'his', 'movie', ',', 'takes', 'a', 'long', 'sober', 'look', 'at', 'what', 'he', 'has', 'wrought', ',', 'and', 'burns', 'it', '.', "''"], 'n

As you can see, this dataset contains `tokens`, `ner_tags` and a `fine_ner_tags` columns. Let's have a look at which labels these last two represent using the [Dataset features](https://huggingface.co/docs/datasets/about_dataset_features).

In [4]:
labels = dataset["train"].features["ner_tags"].feature.names
print(labels)

['O', 'art', 'building', 'event', 'location', 'organization', 'other', 'person', 'product']


In [5]:
fine_labels = dataset["train"].features["fine_ner_tags"].feature.names
print(fine_labels)

['O', 'art-broadcastprogram', 'art-film', 'art-music', 'art-other', 'art-painting', 'art-writtenart', 'building-airport', 'building-hospital', 'building-hotel', 'building-library', 'building-other', 'building-restaurant', 'building-sportsfacility', 'building-theater', 'event-attack/battle/war/militaryconflict', 'event-disaster', 'event-election', 'event-other', 'event-protest', 'event-sportsevent', 'location-GPE', 'location-bodiesofwater', 'location-island', 'location-mountain', 'location-other', 'location-park', 'location-road/railway/highway/transit', 'organization-company', 'organization-education', 'organization-government/governmentagency', 'organization-media/newspaper', 'organization-other', 'organization-politicalparty', 'organization-religion', 'organization-showorganization', 'organization-sportsleague', 'organization-sportsteam', 'other-astronomything', 'other-award', 'other-biologything', 'other-chemicalthing', 'other-currency', 'other-disease', 'other-educationaldegree', '

For the purposes of this tutorial, let's stick with the `ner_tags` coarse-grained labels, but I challenge you to modify this Notebook to train for the fine labels. For the SpanMarker model, any dataset can be used as long as it has a `tokens` and a `ner_tags` column. The `ner_tags` can be annotated using the IOB, IOB2, BIOES or BILOU labeling scheme, but also regular unschemed labels like in this FewNERD example can be used.

### Initializing a `SpanMarkerModel`
A SpanMarker model is initialized via [SpanMarkerModel.from_pretrained](https://tomaarsen.github.io/SpanMarkerNER/api/span_marker.modeling.html#span_marker.modeling.SpanMarkerModel.from_pretrained). This method will be familiar to those who know 🤗 Transformers. It accepts either a path to a local model or the name of a model on the [Hugging Face Hub](https://huggingface.co/models).

Importantly, the model can *either* be an encoder or an already trained and saved SpanMarker model. As we haven't trained anything yet, we will use an encoder. To learn how to load and use a saved SpanMarker model, please have a look at the [Loading & Inferencing](model_loading.ipynb) notebook.

Reasonable options for encoders include BERT, RoBERTa, etc., which means that the following are all good options: `"bert-base-cased"`, `"bert-large-cased"`, `"roberta-base"`, `"roberta-large"`. Not all encoders work though, they **must** allow for `position_ids` as an input argument, which disqualifies DistilBERT, T5, DistilRoBERTa, ALBERT & BART. Furthermore, using uncased models is generally not recommended, as the capitalisation can be very useful to find named entities.

We'll use `"bert-base-cased"` for this notebook. If you're running this on Google Colab, be sure to set hardware accelerator to "GPU" in `Runtime` > `Change runtime type`.

In [6]:
from span_marker import SpanMarkerModel

model_name = "bert-base-cased"
model = SpanMarkerModel.from_pretrained(model_name, labels=labels, model_max_length=256)

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertModel: ['cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


For us, these warnings are expected, as we are initializing `BertModel` for a new task.

Note that we provided `SpanMarkerModel.from_pretrained` with a list of our labels. This is required when training a new model using an encoder. Furthermore, we can specify some useful configuration parameters from `SpanMarkerConfig`, such as:

* `model_max_length`: The maximum number of tokens that the model will process. If you only use short sentences for your model, reducing this number may help training and inference speeds with no loss in performance. Defaults to the encoder maximum, or 512 if the encoder doesn't have a maximum.
* `entity_max_length`: The total number of words that one entity can be. Defaults to 8.

### Training
At this point, our model is already ready for training! We can import [TrainingArguments](https://huggingface.co/docs/transformers/main_classes/trainer#transformers.TrainingArguments) directly from 🤗 Transformers as well as the SpanMarker `Trainer`. The `Trainer` is a subclass of the 🤗 Transformers [Trainer](https://huggingface.co/docs/transformers/main_classes/trainer) that simplifies some tasks for you, but otherwise it works just like the regular `Trainer`.

This next snippet shows some reasonable defaults. Feel free to adjust the batch size to a lower value if you experience out of memory exceptions.

In [7]:
from transformers import TrainingArguments

args = TrainingArguments(
    output_dir="models/span-marker-bert-base-fewnerd-coarse-super",
    learning_rate=5e-5,
    gradient_accumulation_steps=2,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    num_train_epochs=1,
    evaluation_strategy="steps",
    save_strategy="steps",
    eval_steps=200,
    push_to_hub=False,
    logging_steps=50,
    warmup_ratio=0.1,
    dataloader_num_workers=2,
)

Now we can create a SpanMarker `Trainer` in the same way that you would initialize a 🤗 Transformers `Trainer`. We'll train on a subsection of the data to save us some time. Amazingly, this `Trainer` will automatically create logs using exactly the logging tools that you have installed. With other words, if you prefer logging with [Tensorboard](https://www.tensorflow.org/tensorboard), all that you have to do is install it.

In [8]:
from span_marker import Trainer

trainer = Trainer(
    model=model,
    args=args,
    train_dataset=dataset["train"].select(range(8000)),
    eval_dataset=dataset["validation"].select(range(2000)),
)

Let's start training!

In [9]:
trainer.train()

This SpanMarker model will ignore 0.339050% of all annotated entities in the train dataset. This is caused by the SpanMarkerModel maximum entity length of 8 words.
These are the frequencies of the missed entities due to maximum entity length out of 20351 total entities:
- 24 missed entities with 9 words (0.117930%)
- 15 missed entities with 10 words (0.073706%)
- 14 missed entities with 11 words (0.068793%)
- 7 missed entities with 12 words (0.034396%)
- 5 missed entities with 13 words (0.024569%)
- 2 missed entities with 15 words (0.009828%)
- 1 missed entities with 17 words (0.004914%)
- 1 missed entities with 19 words (0.004914%)
Tracking run with wandb version 0.14.0
Run data is saved locally in ...
Syncing run colorful-leaf-761 to Weights & Biases


{'loss': 0.9012, 'learning_rate': 2.032520325203252e-05, 'epoch': 0.04}
{'loss': 0.0813, 'learning_rate': 4.065040650406504e-05, 'epoch': 0.08}
{'loss': 0.0514, 'learning_rate': 4.8777173913043476e-05, 'epoch': 0.12}
{'loss': 0.0385, 'learning_rate': 4.651268115942029e-05, 'epoch': 0.16}


This SpanMarker model won't be able to predict 0.307515% of all annotated entities in the evaluation dataset. This is caused by the SpanMarkerModel maximum entity length of 8 words.
These are the frequencies of the missed entities due to maximum entity length out of 5203 total entities:
- 5 missed entities with 9 words (0.096098%)
- 5 missed entities with 10 words (0.096098%)
- 2 missed entities with 11 words (0.038439%)
- 1 missed entities with 12 words (0.019220%)
- 3 missed entities with 13 words (0.057659%)


{'eval_loss': 0.03596973791718483, 'eval_overall_precision': 0.6802749427202666, 'eval_overall_recall': 0.6297724643270344, 'eval_overall_f1': 0.6540502653449485, 'eval_overall_accuracy': 0.9053643208390295, 'eval_runtime': 28.0718, 'eval_samples_per_second': 87.241, 'eval_steps_per_second': 21.837, 'epoch': 0.16}
{'loss': 0.0334, 'learning_rate': 4.42481884057971e-05, 'epoch': 0.2}
{'loss': 0.0306, 'learning_rate': 4.1983695652173914e-05, 'epoch': 0.24}
{'loss': 0.0278, 'learning_rate': 3.971920289855073e-05, 'epoch': 0.29}


Loading cached processed dataset at ...
Loading cached processed dataset at ...


...

{'loss': 0.0197, 'learning_rate': 1.0280797101449275e-05, 'epoch': 0.81}


{'eval_loss': 0.019617434591054916, 'eval_overall_precision': 0.7771473292897672, 'eval_overall_recall': 0.7659082144234477, 'eval_overall_f1': 0.7714868408274256, 'eval_overall_accuracy': 0.937746128262156, 'eval_runtime': 28.2921, 'eval_samples_per_second': 86.561, 'eval_steps_per_second': 21.667, 'epoch': 0.81}
{'loss': 0.0191, 'learning_rate': 8.016304347826086e-06, 'epoch': 0.86}
{'loss': 0.0187, 'learning_rate': 5.751811594202898e-06, 'epoch': 0.9}
{'loss': 0.0202, 'learning_rate': 3.4873188405797104e-06, 'epoch': 0.94}


Loading cached processed dataset at ...
Loading cached processed dataset at ...


{'loss': 0.0221, 'learning_rate': 1.2228260869565218e-06, 'epoch': 0.98}


{'eval_loss': 0.019159900024533272, 'eval_overall_precision': 0.7773279352226721, 'eval_overall_recall': 0.7774778249132279, 'eval_overall_f1': 0.7774028728429576, 'eval_overall_accuracy': 0.9399702095533473, 'eval_runtime': 28.0225, 'eval_samples_per_second': 87.394, 'eval_steps_per_second': 21.875, 'epoch': 0.98}
{'train_runtime': 453.1296, 'train_samples_per_second': 21.667, 'train_steps_per_second': 2.708, 'train_loss': 0.06319850289734186, 'epoch': 1.0}
TrainOutput(global_step=1227, training_loss=0.06319850289734186, metrics={'train_runtime': 453.1296, 'train_samples_per_second': 21.667, 'train_steps_per_second': 2.708, 'train_loss': 0.06319850289734186, 'epoch': 1.0})

And now the final step is to compute the model's performance.

In [10]:
metrics = trainer.evaluate()
metrics

Loading cached processed dataset at ...
Loading cached processed dataset at ...
{'eval_loss': 0.019206691533327103,
 'eval_overall_precision': 0.7758985200845666,
 'eval_overall_recall': 0.7784419591207096,
 'eval_overall_f1': 0.7771681586293194,
 'eval_overall_accuracy': 0.9398477830602543,
 'eval_runtime': 28.0849,
 'eval_samples_per_second': 87.2,
 'eval_steps_per_second': 21.827,
 'epoch': 1.0}

Additionally, we should evaluate using the test set.

In [11]:
trainer.evaluate(dataset["test"].select(range(2000)), metric_key_prefix="test")

This SpanMarker model won't be able to predict 0.285605% of all annotated entities in the evaluation dataset. This is caused by the SpanMarkerModel maximum entity length of 8 words.
These are the frequencies of the missed entities due to maximum entity length out of 5252 total entities:
- 5 missed entities with 9 words (0.095202%)
- 1 missed entities with 10 words (0.019040%)
- 3 missed entities with 11 words (0.057121%)
- 2 missed entities with 12 words (0.038081%)
- 1 missed entities with 13 words (0.019040%)
- 1 missed entities with 17 words (0.019040%)
- 1 missed entities with 19 words (0.019040%)
- 1 missed entities with 40 words (0.019040%)
{'test_loss': 0.019189156591892242,
 'test_overall_precision': 0.769879287219774,
 'test_overall_recall': 0.7679663608562691,
 'test_overall_f1': 0.7689216342933691,
 'test_overall_accuracy': 0.938544749464231,
 'test_runtime': 28.0932,
 'test_samples_per_second': 86.854,
 'test_steps_per_second': 21.713,
 'epoch': 1.0}

Let's try the model out with some predictions. For this we can use the `model.predict` method, which accepts either:

* A sentence as a string.
* A tokenized sentence as a list of strings.
* A list of sentences as a list of strings.
* A list of tokenized sentences as a list of lists of strings.

The method returns a list of dictionaries for each sentence, with the following keys:

* `"label"`: The string label for the found entity.
* `"score"`: The probability score indicating the model its confidence.
* `"span"`: The entity span as a string.
* `"word_start_index"` and `"word_end_index"`: Integers useful for indexing the entity from a tokenized sentence.
* `"char_start_index"` and `"char_end_index"`: Integers useful for indexing the entity from a string sentence.

In [12]:
sentences = [
    "The Ninth suffered a serious defeat at the Battle of Camulodunum under Quintus Petillius Cerialis in the rebellion of Boudica (61), when most of the foot-soldiers were killed in a disastrous attempt to relieve the besieged city of Camulodunum (Colchester).",
    "He was born in Wellingborough, Northamptonshire, where he attended Victoria Junior School, Westfield Boys School and Sir Christopher Hatton School.",
    "Nintendo continued to sell the revised Wii model and the Wii Mini alongside the Wii U during the Wii U's first release year.",
    "Dorsa has a Bachelor of Music in Composition from California State University, Northridge in 2001, Master of Music in Harpsichord Performance at Cal State Northridge in 2004, and a Doctor of Musical Arts at the University of Michigan, Ann Arbor in 2008."
]

entities_per_sentence = model.predict(sentences)

for entities in entities_per_sentence:
    for entity in entities:
        print(entity["span"], "=>", entity["label"])
    print()

Battle of Camulodunum => event
Quintus Petillius Cerialis => person
Boudica => person

Wellingborough => location
Northamptonshire => location
Victoria Junior School => organization
Westfield Boys School => organization
Sir Christopher Hatton School => organization

Nintendo => organization
Wii => product
Wii Mini => product
Wii U => product
Wii U => product

Dorsa => person
Bachelor of Music in Composition => other
California State University => organization
Northridge => location
Cal State Northridge => organization
Ann Arbor => organization



Very impressive performance for less than 8 minutes trained! 🎉

Once trained, we can save our new model locally.

In [13]:
trainer.save_model("models/span-marker-bert-base-fewnerd-coarse-super/checkpoint-final")

Or we can push it to the 🤗 Hub like so. I've commented it away for now to prevent people from accidentally pushing models.

In [14]:
trainer.push_to_hub()

If we want to use it again, we can just load it using the checkpoint or using the model name on the Hub. This is how it would be done using a local checkpoint.

In [15]:
# model = SpanMarkerModel.from_pretrained("models/span-marker-bert-base-fewnerd-coarse-super/checkpoint-final")

That was all! As simple as that. If we put it all together into a single script, it looks something like this:
```python
from datasets import load_dataset
from span_marker import SpanMarkerModel, Trainer
from transformers import TrainingArguments

def main():
    dataset = load_dataset("DFKI-SLT/few-nerd", "supervised")
    labels = dataset["train"].features["ner_tags"].feature.names

    model_name = "bert-base-cased"
    model = SpanMarkerModel.from_pretrained(model_name, labels=labels)

    args = TrainingArguments(
        output_dir="models/span-marker-bert-base-fewnerd-coarse-super",
        learning_rate=5e-5,
        gradient_accumulation_steps=2,
        per_device_train_batch_size=4,
        per_device_eval_batch_size=4,
        num_train_epochs=1,
        evaluation_strategy="steps",
        save_strategy="steps",
        eval_steps=200,
        push_to_hub=False,
        logging_steps=50,
        warmup_ratio=0.1,
        dataloader_num_workers=2,
    )

    trainer = Trainer(
        model=model,
        args=args,
        train_dataset=dataset["train"].select(range(8000)),
        eval_dataset=dataset["validation"].select(range(2000)),
    )

    trainer.train()
    trainer.save_model("models/span-marker-bert-base-fewnerd-coarse-super/checkpoint-final")

    metrics = trainer.evaluate()
    print(metrics)

if __name__ == "__main__":
    main()
```

With `wandb` initialized, you can enjoy their very useful training graphs straight in your browser. It ends up looking something like this.
![image](https://user-images.githubusercontent.com/37621491/235196250-15d595f4-6d72-4625-bde9-f3783484997b.png)
![image](https://user-images.githubusercontent.com/37621491/235196335-6f36a7fb-5274-4ce5-a1f3-1d2ad35b26a4.png)

Furthermore, you can use the `wandb` hyperparameter search functionality using the tutorial from the Hugging Face documentation [here](https://huggingface.co/docs/transformers/hpo_train). This transfers very well to the SpanMarker `Trainer`.